# Feature - Dataset

> Fill in a module description here

In [ ]:
#| default_exp feature.dataset

In [ ]:
#| importi
from dataclasses import dataclass, field

from typing import  List

import mbison.client.core as dmda

In [ ]:
#| hide
import nbdev; 

import os

## Datasets

In [ ]:
#| export 
def get_datasets(auth : dmda.DomoAuth, debug_api : bool = False, return_raw: bool = False):
    endpoint = '/api/data/ui/v3/datasources/search'

    payload = {
            "entities": ["DATASET"],
            "filters": [],
            "combineResults":"true",
            "query":"*",
            "count":1000,
            "offset":0,
            "sort":{"isRelevance":"false",
                    "fieldSorts":[{"field":"create_date",
                                    "sortOrder":"DESC"}]}}
    
    res = dmda.domo_api_request(
        auth = auth,
        endpoint= endpoint,
        request_type='POST',
        debug_api= debug_api,
        body = payload
        
    )

    if return_raw:
        return res
    
    res.response = res.response['dataSources']
    
    return res

In [ ]:
auth = dmda.DomoAuth(domo_instance= os.environ['DOMO_INSTANCE'],
                access_token = os.environ['DOMO_ACCESS_TOKEN']
                )


res = get_datasets(auth = auth, debug_api= False)

datasets = res.response
datasets[0:1]

*** Using Access Token Auth


[{'id': '398677b7-ca3a-4bc5-9f4d-3a579278b71d',
  'displayType': 'webform',
  'dataProviderType': 'webform',
  'type': 'webform',
  'name': 'Example Sales Data',
  'owner': {'id': '1351541814',
   'name': 'jae wilson',
   'type': 'USER',
   'group': False},
  'status': 'VALID',
  'created': 1714506702000,
  'lastUpdated': 1714506705068,
  'dataLastUpdated': 0,
  'rowCount': 19378,
  'columnCount': 5,
  'cardInfo': {'cardCount': 1, 'cardViewCount': 0},
  'state': 'VALID',
  'validConfiguration': True,
  'validAccount': True,
  'transportType': 'WEBFORM',
  'adc': False,
  'adcExternal': False,
  'cloudId': 'domo',
  'cloudName': 'Domo',
  'hidden': False,
  'tagsList': [],
  'schema': {'name': '398677b7-ca3a-4bc5-9f4d-3a579278b71d',
   'tables': [{'columns': [{'name': 'date_ymd',
       'id': 'date_ymd',
       'type': 'DATE',
       'visible': True,
       'order': 0},
      {'name': 'revenue',
       'id': 'revenue',
       'type': 'DOUBLE',
       'visible': True,
       'order': 0},

### PDP

In [ ]:
#|export 

def get_pdp_policies(auth : dmda.DomoAuth, datasetId):
        endpoint = f'/api/query/v1/data-control/{datasetId}/filter-groups'
        
        params = {"options": "load_associations,load_filters,include_open_policy"}

        response = dmda.domo_api_request(endpoint=endpoint,
                                    auth = auth, request_type= 'GET',
                                    params=params
                                    ) 
        
        return response

def get_pdp_policy(auth: dmda.DomoAuth, datasetId, policy_id):
        endpoint = f'/api/query/v1/data-control/{datasetId}/filter-groups/{policy_id}'

        res = dmda.domo_api_request(
                auth = auth,
                method = 'GET',
                endpoint = endpoint,
                params = {'options':'load_associations,load_filters,include_open_policy'})
        
        return res

In [ ]:
get_pdp_policies(auth = auth, datasetId=datasets[0]['id'])

RsponseGetData(response=[{'name': 'All Rows', 'filterGroupId': 7, 'dataSourceId': '398677b7-ca3a-4bc5-9f4d-3a579278b71d', 'type': 'open', 'dataSourcePermissions': False, 'order': 0}], is_success=True, status=200)

In [ ]:
# | export


def create_pdp_policy(auth: dmda.DomoAuth, datasetId: str, pdp_object: dict):

    endpoint = f"/api/query/v1/data-control/{datasetId}/filter-groups"

    res = dmda.domo_api_request(
        auth=auth, request_type="POST", endpoint=endpoint, body=pdp_object
    )

    return res


def update_pdp_policy(auth: dmda.DomoAuth, datasetId, policyId, pdp_object: dict):
    endpoint = f"/api/query/v1/data-control/{datasetId}/filter-groups/{policyId}"

    res = dmda.domo_api_request(
        auth=auth,
        request_type="PUT",
        endpoint=endpoint,
        body=pdp_object,
    )

    return res


def delete_pdp_policy(auth: dmda.DomoAuth, datasetId, policyId):
    endpoint = f"/api/query/v1/data-control/{datasetId}/filter-groups/{policyId}"

    res = dmda.domo_api_request(auth=auth, endpoint=endpoint, request_type="DELETE")

    return res

### DomoDataset

In [ ]:
# | export


@dataclass
class DomoDatasetPolicy:
    name: str


@dataclass
class DomoDataset:
    id: str
    name: str

    auth: dmda.DomoAuth

    pdp_policies: List[dict] = None
    domo_policies: List[DomoDatasetPolicy] = None

    @classmethod
    def _from_json(cls, auth: dmda.DomoAuth, obj: dict):
        return cls(auth=auth, id=obj["id"], name=obj["name"])

    def get_pdp_policies(self, return_raw: bool = False):

        res = get_pdp_policies(auth=self.auth, datasetId=self.id)

        if return_raw:
            return res

        self.pdp_policies = res.response
        self.domo_policies = [
            DomoDatasetPolicy(name=obj["name"]) for obj in self.pdp_policies
        ]

        return self.domo_policies

### DomoDatasets

In [ ]:
#| export 

@dataclass
class DomoDatasets:
    auth: dmda.DomoAuth
    raw_datasets: List[dict] = field(default_factory=lambda: [])

    domo_datasets: List[DomoDataset] = field(default_factory=lambda: [])

    def get_datasets(self, debug_api : bool = False, return_raw: bool = False):

        res = get_datasets(self.auth, debug_api = debug_api)

        if return_raw: 
            return res
        
        self.raw_datasets = res.response

        self.domo_datasets = [
            DomoDataset._from_json(obj=obj, 
                                   auth=self.auth) for obj in self.raw_datasets
        ]

        return self.domo_datasets

    def print_datasets(self):
        print(self.domo_datasets)

In [ ]:
domo_datasets = DomoDatasets(auth=auth)
domo_datasets.get_datasets()

[DomoDataset(id='398677b7-ca3a-4bc5-9f4d-3a579278b71d', name='Example Sales Data', auth=DomoAuth(domo_instance='datacrew-space', username=None), pdp_policies=None, domo_policies=None),
 DomoDataset(id='38b145a6-297a-44b7-9507-94f1f3cb8e67', name='Example Sales', auth=DomoAuth(domo_instance='datacrew-space', username=None), pdp_policies=None, domo_policies=None),
 DomoDataset(id='bdc28de2-58f1-4291-9436-bbc5ae793861', name='DST_TST_domo_api', auth=DomoAuth(domo_instance='datacrew-space', username=None), pdp_policies=None, domo_policies=None),
 DomoDataset(id='28ea5493-e20c-47f6-9235-06d9624d31e2', name='Pricing product details', auth=DomoAuth(domo_instance='datacrew-space', username=None), pdp_policies=None, domo_policies=None),
 DomoDataset(id='a0e82f76-3006-4025-b7f7-cdd0c2291f63', name='BF Sales by Hour', auth=DomoAuth(domo_instance='datacrew-space', username=None), pdp_policies=None, domo_policies=None),
 DomoDataset(id='d32743ca-3c47-4d68-aaea-ff6f374727ac', name='Transaction produ

In [ ]:
domo_datasets.domo_datasets[0].get_pdp_policies()

[DomoDatasetPolicy(name='All Rows')]

In [ ]:
#| hide
nbdev.nbdev_export('datasets.ipynb')